In [49]:
# 創建 Table
#cur.execute("CREATE TABLE results(jid, date, law, keyword_type, keywords)")
#con.commit()

In [50]:
from ElementExtractor import Judicial_Parser
import os, time, traceback, json, re

In [ ]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [7]:
# To use GPU:
#    1. Install tensorflow-gpu (see Installation)
#    2. Set CUDA_VISIBLE_DEVICES environment variable, e.g. os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#    3. Set disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)
# To use CPU:
ws = WS("./data", disable_cuda=False)
pos = POS("./data", disable_cuda=False)
ner = NER("./data", disable_cuda=False)

C:\ProgramData\Anaconda3\envs\ckipcore\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\ProgramData\Anaconda3\envs\ckipcore\lib\site-packages\ckiptagger\model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\ProgramData\Anaconda3\envs\ckipcore\lib\site-packages\ckiptagger\model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.n

In [6]:
# Downloads to ./data.zip (2GB) and extracts to ./data/
# data_utils.download_data_url("./") # iis-ckip
#data_utils.download_data_gdown("./") # gdrive-ckip

Downloading...
From (original): https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
From (redirected): https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771&confirm=t&uuid=06dbc6eb-cbd8-4abe-8206-d345b3ad7aab
To: C:\課程\交大\論文\Code\data.zip
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.88G/1.88G [03:01<00:00, 10.3MB/s]


In [8]:
# 簡繁轉換器 (暫時用不到)
# import opencc
# converter = opencc.OpenCC('t2s.json')

In [46]:
import sqlite3
con = sqlite3.connect("JudicialAnalysis.db")

In [47]:
cur = con.cursor()

In [64]:
from IPython.display import clear_output

source_file = os.path.join(
        'C:\\', '課程', '交大','論文','案件整理',
        'json_files.txt'
        )
    
count_file_done_num = 0
clear_output()
print('-'*50)
print(f'>>開始分析...')
print(f'[當前已分析數量] {count_file_done_num}')

with open(source_file, 'r', encoding='utf-8') as f:
    # 2022~2023年判決書有211萬7,454個json檔
    # 故一次讀一個處理，減少記憶體占用
    raw_data = ''
    
    while 1: 
        # 開始逐案讀取資料分析
        file = f.readline()
        count_file_done_num += 1
        
        if '刑事' in file:
            # 篩掉一些比較非刑案的
            if not ('簡易' in file or '民事' in file or '補償' in file or '憲法' in file or '商業' in file):
                # 開始讀取裁判書內容
                file = file.strip()
                
                # 顯示進度
                clear_output()
                print('-'*50)
                print(f'>>開始分析...')
                print(f'[當前已分析數量] {count_file_done_num}')
                print(f'[當前檔案] {file}')
                
                with open(file, 'r', encoding='utf-8') as jf:
                    json_data = json.load(jf)
                
                # 資料清洗
                parser = Judicial_Parser(json_data)
                
                jid = parser.jid
                date = parser.date
                raw = parser.raw
                text = parser.text
                text_u = parser.text_u
                
                # 確定是否已經在資料庫內
                check_res = cur.execute(f'SELECT jid FROM results WHERE jid="{jid}"')
                temp = check_res.fetchone()
                if temp:
                    print(f'[JID] {jid}')
                    print('>>此JID發現資料庫已有，跳過')
                    continue
                    
                # 沒有在資料庫內才處理
                # Input text
                print('[WS] 分析中')
                sentence_list = text_u.split('。')
                word_sentence_list = ws(
                    sentence_list,
                    # sentence_segmentation = True, # To consider delimiters
                    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
                    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
                    # coerce_dictionary = dictionary2, # words in this dictionary are forced
                )
                print('[POS] 分析中')
                pos_sentence_list = pos(word_sentence_list)
                print('[NER] 分析中')
                entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
                
                # 用 POS 區分的類型
                type_dict = dict()
                for idx, words in enumerate(word_sentence_list):
                    for idx2, w in enumerate(words):
                        temp_type = pos_sentence_list[idx][idx2]
                        type_dict.setdefault(temp_type, set())
                        type_dict[temp_type].add(w)
                
                # 經人工檢查，N(名詞)比較符合需要的有
                # 'Na', 'Nb'
                type_N = [t for t in type_dict.keys() if t.startswith('N')]
                type_N.sort()
                for n in type_N:
                    print('-'*50)
                    print(f'TYPE: {n}')
                    print(type_dict[n])

                # 經人工檢查，V(動詞)比較符合需要的有
                # 'VA', 'VC', 'VD, 'VH', 
                type_V = [t for t in type_dict.keys() if t.startswith('V')]
                type_V.sort()
                for v in type_V:
                    print('-'*50)
                    print(f'TYPE: {v}')
                    print(type_dict[v])
                
                # 擷取法條
                laws = set()
                for e in entity_sentence_list:
                    for ee in e:
                        if ee[2] == 'LAW':
                            laws.add(ee[3])
                print('-'*50)
                print('TYPE: LAW')
                print(laws)
                
                if not laws:
                    print('>>此篇未發現 law 犯罪條文，跳過')
                    continue
                
                # 寫入 Table
                print('-'*50)
                print('>>準備寫入 Table')
                # results(jid, date, law, keyword_type, keywords)
                
                for law in laws:
                    # 先從名詞
                    print('-'*50)
                    print('>>準備寫入 Na')
                    if 'Na' in type_dict:
                        na_set = {n.strip().replace(' ', '') for n in type_dict['Na'] if len(n.strip().replace(' ', '')) > 1}
                        na_strs = ','.join(na_set)
                        cur.execute(f"INSERT INTO results VALUES('{jid}', '{date}', '{law}', 'Na', '{na_strs}')")
                        con.commit()
                    
                    # 先從名詞
                    print('-'*50)
                    print('>>準備寫入 Nb')
                    if 'Nb' in type_dict:
                        na_set = {n.strip().replace(' ', '') for n in type_dict['Nb'] if len(n.strip().replace(' ', '')) > 1}
                        na_strs = ','.join(na_set)
                        cur.execute(f"INSERT INTO results VALUES('{jid}', '{date}', '{law}', 'Nb', '{na_strs}')")
                        con.commit()
                    
                    # 再來動詞
                    print('-'*50)
                    print('>>準備寫入 VA')
                    if 'VA' in type_dict:
                        na_set = {n.strip().replace(' ', '') for n in type_dict['VA'] if len(n.strip().replace(' ', '')) > 1}
                        na_strs = ','.join(na_strs)
                        cur.execute(f"INSERT INTO results VALUES('{jid}', '{date}', '{law}', 'VA', '{na_strs}')")
                        con.commit()
                    
                    # 再來動詞
                    print('-'*50)
                    print('>>準備寫入 VC')
                    if 'VC' in type_dict:
                        na_set = {n.strip().replace(' ', '') for n in type_dict['VC'] if len(n.strip().replace(' ', '')) > 1}
                        na_strs = ','.join(na_set)
                        cur.execute(f"INSERT INTO results VALUES('{jid}', '{date}', '{law}', 'VC', '{na_strs}')")
                        con.commit()
                    
                    # 再來動詞
                    print('-'*50)
                    print('>>準備寫入 VD')
                    if 'VD' in type_dict:
                        na_set = {n.strip().replace(' ', '') for n in type_dict['VD'] if len(n.strip().replace(' ', '')) > 1}
                        na_strs = ','.join(na_set)
                        cur.execute(f"INSERT INTO results VALUES('{jid}', '{date}', '{law}', 'VD', '{na_strs}')")
                        con.commit()
                    
                    # 再來動詞
                    print('-'*50)
                    print('>>準備寫入 VH')
                    if 'VH' in type_dict:
                        na_set = {n.strip().replace(' ', '') for n in type_dict['VH'] if len(n.strip().replace(' ', '')) > 1}
                        na_strs = ','.join(na_set)
                        cur.execute(f"INSERT INTO results VALUES('{jid}', '{date}', '{law}', 'VH', '{na_strs}')")
                        con.commit()

                    print('-'*50)
                    print('>>完成!!!')

--------------------------------------------------
>>開始分析...
[當前已分析數量] 4190
[當前檔案] C:\課程\交大\論文\案件整理\202201\最高法院刑事\TPSM,110,台上,1492,20220127,1.json
[WS] 分析中
[POS] 分析中
[NER] 分析中
--------------------------------------------------
TYPE: Na
{'上訴人', '審卷', '重訴字', '共犯', '客戶', '金額', '存款', '刑事', '犯行', '語', '案件', '卷', '職責', '經驗', '旨', '罪疑', '主文', '證人', '證明力', '銀行法', '論理', '資料', '規定', '自白', '證明', '起訴書', '事', '交易', '業務', '綽號', '彙整表', '人', '程式', '罪刑', '原審', '前述', '情形', '審判長', '總額', '字', '意見', '職權', '要件', '訴訟法', '核係', '責', '人民幣', '答辯狀', '法令', '人稱', '匯兌', '帳戶', '理由', '附表', '律師', '男子', '事實', '犯意', '條件', '事證', '共同犯', '附件', '新臺幣', '檢察官', '成本', '供證', '地方', '情事', '辯護人', '起訴案號', '法則', '總金額', '書狀', '財物', '核', '判長', '書記官', '審法官', '心證', '證據', '事實欄', '正本', '報酬', '法律', '偵字', '結果', '所得', '法官'}
--------------------------------------------------
TYPE: Nb
{'蔡東德', '侯 廷', '葉姿紋', '林嘉進', '林 恆', '曾郁榮', '汪堯雄', '黃啟菖', '林 海', ' 昌 \n', '洪雪嬌', '金上', '徐 昌', ' 吉 ', '錦', '周 政', '曾士豪', '阿平', ' 祥', '黃月伶'}
-------------------------

KeyError: 'VD'

In [11]:
con.close()